In [1]:
# import the required packages
from ema_workbench import (RealParameter,
                           CategoricalParameter,
                           TimeSeriesOutcome,
                           Policy,
                           MultiprocessingEvaluator,
                           SequentialEvaluator, 
                           ema_logging)

from linnyr_connector import LinnyRModel_Botlek

import os

# enable info logging
ema_logging.log_to_stderr(ema_logging.INFO)

# define the model
model = LinnyRModel_Botlek(name = 'BotlekModel', wd = os.getcwd(), model_file = 'botlek_model.lnr')

# define the uncertain factors
model.uncertainties = [RealParameter(name = 'Electricity price in 2030 (€/MWh)',
                                     variable_name = 'E day-ahead:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'Gas price in 2030 (€/Nm3)',
                                     variable_name = 'natural gas market:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'CO2 emission price in 2030 (€/ton)', # negative values
                                     variable_name = 'CO2 EUROPEAN EMISSION ALLOWANCES:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'Hydrogen price in 2030 (€/Nm3)',
                                     variable_name = 'H2 markt:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'NaOH 50% price in 2030 (€/ton)',
                                     variable_name = 'NaOH 50%:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'Factor upward balancing electricity price (€/MWh)',
                                     variable_name = 'Unbal opregelen:Price',
                                     lower_bound = 0.7, 
                                     upper_bound = 1.3),
                       RealParameter(name = 'Factor downward balancing electricity price (€/MWh)',
                                     variable_name = 'Unbal afregelen:Price',
                                     lower_bound = 0.7, 
                                     upper_bound = 1.3),
                       RealParameter(name = 'Factor electricity supply imbalance market (MWh)',
                                     variable_name = 'Unbal afregelen:LB',
                                     lower_bound = 0.7, 
                                     upper_bound = 1.3),
                       RealParameter(name = 'Factor electricity demand imbalance market (MWh)',
                                     variable_name = 'Unbal opregelen:UB',
                                     lower_bound = 0.7, 
                                     upper_bound = 1.3),
                       RealParameter(name = 'E-boiler CAPEX (€/MW)', # per quarter
                                     variable_name = 'Capex E-boiler:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'E-boiler OPEX (€/MW/year)', # per quarter
                                     variable_name = 'OPEX E-BOILER:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0),
                       RealParameter(name = 'Steam Pipe CAPEX (€)',
                                     variable_name = 'CAPEX Steam Pipe:Price',
                                     lower_bound = 5.0, 
                                     upper_bound = 20.0)]

# define the levers (Power-to-X alternatives)
model.levers = [CategoricalParameter(name = 'Steam Pipe to Nouryon',
                                     variable_name = 'Option: transport to Nouryon (Steam pipe owner):UB',
                                     categories = [0, 7.5]),
                CategoricalParameter(name = 'Steam Pipe to Air Liquide and Huntsman',
                                     variable_name = 'FUTURE: transport 5210 site (Steam pipe owner):UB',
                                     categories = [0, 30]),
                CategoricalParameter(name = 'E-boiler 1',
                                     variable_name = 'Electrode boiler 50 bar 2/7 aFRR (Air Liquide):UB',
                                     categories = [0, 5]),
                CategoricalParameter(name = 'E-boiler 2',
                                     variable_name = 'electrode boiler 50 bar 5/7 inzetbaar (Air Liquide):UB',
                                     categories = [0, 17.5]),
                CategoricalParameter(name = 'E-boiler 3',
                                     variable_name = 'by-pass aFRR (ghost actor):UB',
                                     categories = [1000, 0]),
                CategoricalParameter(name = 'E-boiler 4',
                                     variable_name = 'AL 50 bar fixed rate (Air Liquide):LB',
                                     categories = [0, 22.5]),
                CategoricalParameter(name = 'E-boiler 5',
                                     variable_name = 'DA inkoop EB70 (Air Liquide):UB',
                                     categories = [0, 6]),
                CategoricalParameter(name = 'Chlorine storage',
                                     variable_name = 'stored CL2 (Nouryon):UB',
                                     categories = [1600, 3200])]

# define the outcomes
model.outcomes = [TimeSeriesOutcome(name = 'CF total',
                                    variable_name = 'CF total' ),
                  TimeSeriesOutcome(name = 'CO2 emissions',
                                    variable_name = 'CO2 emission')]

# define a function for generating policies
def generate(name, steam_pipe, e_boiler, chorine_storage):
    
    d = {}
    
    if steam_pipe == True:
        d['Steam Pipe to Nouryon'] = 7.5
        d['Steam Pipe to Air Liquide and Huntsman'] = 30
    else:
        d['Steam Pipe to Nouryon'] = 0
        d['Steam Pipe to Air Liquide and Huntsman'] = 0
    if e_boiler == True:
        d['E-boiler 1'] = 5
        d['E-boiler 2'] = 17.5
        d['E-boiler 3'] = 0
        d['E-boiler 4'] = 22.5
        d['E-boiler 5'] = 6
    else:
        d['E-boiler 1'] = 0
        d['E-boiler 2'] = 0
        d['E-boiler 3'] = 1000
        d['E-boiler 4'] = 0
        d['E-boiler 5'] = 0
    if chorine_storage == True:
        d['Chlorine storage'] = 3200
    else:
        d['Chlorine storage'] = 1600
        
    return Policy(name, **d)

# generate the desired policies
policies = [generate(name = 'None', steam_pipe = False, e_boiler = False, chorine_storage = False),
            generate(name = 'Only Steam Pipe', steam_pipe = True, e_boiler = False, chorine_storage = False),
            generate(name = 'Only E-boiler', steam_pipe = False, e_boiler = True, chorine_storage = False),
            generate(name = 'Only Chlorine storage', steam_pipe = False, e_boiler = False, chorine_storage = True),
            generate(name = 'Steam Pipe & E-boiler', steam_pipe = True, e_boiler = True, chorine_storage = False),
            generate(name = 'Steam Pipe & Chlorine storage', steam_pipe = True, e_boiler = False, chorine_storage = True),
            generate(name = 'E-boiler & Chlorine storage', steam_pipe = False, e_boiler = True, chorine_storage = True),
            generate(name = 'All options', steam_pipe = True, e_boiler = True, chorine_storage = True)]

# run the analysis
with SequentialEvaluator(model) as evaluator:
     experiments, outcomes = evaluator.perform_experiments(policies = policies, scenarios = 1)

[MainProcess/INFO] performing 1 scenarios * 8 policies * 1 model(s) = 8 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] experiments finished


In [2]:
# changes the names of the uncertain factors to real name
loc = experiments.columns.get_loc("scenario")
names = experiments.columns[:loc]
experiments.drop(columns = names, inplace =  True)
for i in experiments.columns[3:]:
    loc = experiments.columns.get_loc(i) - 3
    experiments.rename(columns={i : names[loc]}, inplace = True)
    
# drop the 'model' column
experiments.drop(columns = 'model', inplace = True)

In [3]:
experiments

,scenario,policy,CO2 emission price in 2030 (€/ton),E-boiler CAPEX (€/MW),E-boiler OPEX (€/MW/year),Electricity price in 2030 (€/MWh),Factor downward balancing electricity price (€/MWh),Factor electricity demand imbalance market (MWh),Factor electricity supply imbalance market (MWh),Factor upward balancing electricity price (€/MWh),Gas price in 2030 (€/Nm3),Hydrogen price in 2030 (€/Nm3),NaOH 50% price in 2030 (€/ton),Steam Pipe CAPEX (€)
0,0,None,9.419865,18.977845,9.658836,9.854178,6.424374,1.086647,1.166373,1.058325,0.86525,16.152956,16.585776,15.583603
1,0,Only Steam Pipe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Only E-boiler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Only Chlorine storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Steam Pipe & E-boiler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,Steam Pipe & Chlorine storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,E-boiler & Chlorine storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,All options,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
outcomes

{'CF total': array([[19957.179688, 20033.417969, 19882.267578, 19835.027344,
         20173.560547, 20175.916016, 20266.603516, 19810.761719,
         19817.064453, 20108.873047, 19996.15625 , 19903.767578,
         19978.134766, 20226.039063, 20016.107422, 20120.966797,
         20086.505859, 20128.945313, 19880.078125, 19886.378906,
         20150.400391, 20089.509766, 19941.158203, 20102.111328,
         20127.933594, 20025.671875, 19740.457031, 19699.484375,
         19749.589844, 19734.375   ],
        [22637.185547, 35078.261719, 19694.818359, 19694.697266,
         19827.757813, 28835.474609, 20061.902344, 19841.513672,
         22167.410156, 22574.503906, 19974.03125 , 19951.242188,
         20008.466797, 32191.330078, 19999.578125, 20085.107422,
         19922.519531, 20167.28125 , 20253.134766, 20202.314453,
         20350.826172, 20014.185547, 20494.166016, 20150.904297,
         19621.689453, 19504.546875, 30328.544922, 19550.291016,
         19152.791016, 19260.470703],
  